In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/Dataset Demo/dataset_test

/content/drive/MyDrive/Colab Notebooks/Dataset Demo/dataset_test


In [2]:
!pip install wandb
#!pip install panda
#!pip install requests
#!pip install request

     |████████████████████████████████| 1.8 MB 4.5 MB/s 
     |████████████████████████████████| 144 kB 49.7 MB/s 
     |████████████████████████████████| 181 kB 49.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=bcbbf481d2e61e59580493ead05ba1befc9d112393d728c9a2f5c66c2a630868
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [3]:
import torch
import numpy as np
#import panda as pd
import wandb
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image

In [4]:
import label_utils

In [5]:
wandb.login()
config = {
    "num_workers": 4,
    "pin_memory": True,
    "batch_size": 32,
    "dataset": "drinks",
    "train_split": "drinks/labels_train.csv",
    "test_split": "drinks/labels_test.csv",}
run = wandb.init(project="dataloader-project", entity="josedolot", config=config)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: josedolot (use `wandb login --relogin` to force relogin)


In [6]:
test_dict, test_classes = label_utils.build_label_dictionary(
    config['test_split'])
train_dict, train_classes = label_utils.build_label_dictionary(
    config['train_split'])
test_dict_coco = label_utils.build_label_dictionary_coco(
    config['test_split'])
train_dict_coco = label_utils.build_label_dictionary_coco(
    config['train_split'])


class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dictionary, transform=None):
        self.dictionary = dictionary
        self.transform = transform

    def __len__(self):
        return len(self.dictionary)

    def __getitem__(self, idx):
        # retrieve the image filename
        key = list(self.dictionary.keys())[idx]
        # retrieve all bounding boxes
        boxes = self.dictionary[key]
        # open the file as a PIL image
        img = Image.open(key)
        # apply the necessary transforms
        # transforms like crop, resize, normalize, etc
        if self.transform:
            img = self.transform(img)
        
        # return a list of images and corresponding labels
        return img, boxes


train_split = ImageDataset(train_dict, transforms.ToTensor())
test_split = ImageDataset(test_dict, transforms.ToTensor())

# This is approx 95/5 split
print("Train split len:", len(train_split))
print("Test split len:", len(test_split))

# We do not have a validation split

def collate_fn(batch):
    maxlen = max([len(x[1]) for x in batch])
    images = []
    boxes = []
    for i in range(len(batch)):
        img, box = batch[i]
        images.append(img)
        # pad with zeros if less than maxlen
        if len(box) < maxlen:
            box = np.concatenate(
                (box, np.zeros((maxlen-len(box), box.shape[-1]))), axis=0)

        box = torch.from_numpy(box)
        boxes.append(box)

    return torch.stack(images, 0), torch.stack(boxes, 0)


train_loader = DataLoader(train_split,
                          batch_size=config['batch_size'],
                          shuffle=True,
                          num_workers=config['num_workers'],
                          pin_memory=config['pin_memory'],
                          collate_fn=collate_fn)

test_loader = DataLoader(test_split,
                         batch_size=config['batch_size'],
                         shuffle=False,
                         num_workers=config['num_workers'],
                         pin_memory=config['pin_memory'],
                         collate_fn=collate_fn)

Train split len: 996
Test split len: 51


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [8]:
test_dict_coco

{'0010000.jpg': {'area': [21808.0],
  'boxes': [430.0, 138.0, 524.0, 370.0],
  'image_id': 10000,
  'iscrowd': [0],
  'labels': [2.0]},
 '0010001.jpg': {'area': [20416.0],
  'boxes': [442.0, 149.0, 530.0, 381.0],
  'image_id': 10001,
  'iscrowd': [0],
  'labels': [2.0]},
 '0010002.jpg': {'area': [12390.0],
  'boxes': [399.0, 255.0, 469.0, 432.0],
  'image_id': 10002,
  'iscrowd': [0],
  'labels': [2.0]},
 '0010003.jpg': {'area': [20384.0],
  'boxes': [357.0, 195.0, 448.0, 419.0],
  'image_id': 10003,
  'iscrowd': [0],
  'labels': [2.0]},
 '0010004.jpg': {'area': [20250.0],
  'boxes': [353.0, 153.0, 443.0, 378.0],
  'image_id': 10004,
  'iscrowd': [0],
  'labels': [2.0]},
 '0010005.jpg': {'area': [15200.0],
  'boxes': [329.0, 215.0, 405.0, 415.0],
  'image_id': 10005,
  'iscrowd': [0],
  'labels': [2.0]},
 '0010006.jpg': {'area': [25186.0],
  'boxes': [472.0, 138.0, 570.0, 395.0],
  'image_id': 10006,
  'iscrowd': [0],
  'labels': [2.0]},
 '0010007.jpg': {'area': [34163.0],
  'boxes': [

In [ ]:
test = 0
dict = []
for a in range(31):

    # sample one mini-batch
    images, boxes = next(iter(train_loader))
    # map of label to class name
    class_labels = {i: label_utils.index2class(i) for i in train_classes}

    #run.display(height=1000)
    #table = wandb.Table(columns=['Image'])

    # we use wandb to visualize the objects and bounding boxes
    for image, box in zip(images, boxes):
        #dict = []
        for i in range(box.shape[0]):
            if box[i, -1] == 0:
                continue
            dict_item = {}
            dict_item["boxes"] = {
                "xmin": box[i, 0].item(),
                "ymin": box[i, 2].item(),
                "xmax": box[i, 1].item(),
                "ymax": box[i, 3].item(),
            }
            dict_item["domain"] = "pixel"
            dict_item["class_id"] = (int)(box[i, 4].item())
            dict_item["box_caption"] = label_utils.index2class(
                dict_item["class_id"])
            dict.append(dict_item)
            #print(image)
        test += 1
        print(test, image)

        #img = wandb.Image(image, boxes={
        #    "ground_truth": {
        #        "box_data": dict,
        #        "class_labels": class_labels
        #    }
        #})
        #table.add_data(img)

print(dict)

#wandb.log({"train_loader": table})
wandb.finish()

In [9]:
!git pull


remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/jgdolot/dataset_test
 * [new branch]      v1         -> origin/v1
 * [new branch]      v2         -> origin/v2
Already up to date.
